In [1]:
import requests
import pandas as pd
import numpy as np
import os

import datetime as dt
import pytz

timezone = 'Asia/Taipei'
path_data_base = os.path.join(os.getcwd(), '..', 'Data')

In [2]:
def check_data(df:pd.DataFrame):
    print(f"unique_index? {df.index.is_unique}")
    print(df.reset_index().info())


## 期權資料

### tvapi (海期監控-期天大勝?)
* 大台最遠'2009-1-1', 小台最遠'2020-7-1'
* Tickers
    * 大台TX, 小台MTX

In [44]:
pd.Timedelta?

Init signature:
pd.Timedelta(
    value=<object object at 0x00000172DC1DC520>,
    unit=None,
    **kwargs,
)
Docstring:     
Represents a duration, the difference between two dates or times.

Timedelta is the pandas equivalent of python's ``datetime.timedelta``
and is interchangeable with it in most cases.

Parameters
----------
value : Timedelta, timedelta, np.timedelta64, str, or int
unit : str, default 'ns'
    Denote the unit of the input, if input is an integer.

    Possible values:

    * 'W', 'D', 'T', 'S', 'L', 'U', or 'N'
    * 'days' or 'day'
    * 'hours', 'hour', 'hr', or 'h'
    * 'minutes', 'minute', 'min', or 'm'
    * 'seconds', 'second', or 'sec'
    * 'milliseconds', 'millisecond', 'millis', or 'milli'
    * 'microseconds', 'microsecond', 'micros', or 'micro'
    * 'nanoseconds', 'nanosecond', 'nanos', 'nano', or 'ns'.

**kwargs
    Available kwargs: {days, seconds, microseconds,
    milliseconds, minutes, hours, weeks}.
    Values for construction in compat with da

In [29]:
dt.datetime.now()

datetime.datetime(2024, 1, 11, 11, 39, 25, 259932)

In [46]:
def tvapi(symbol, freq:str, start:str, end:str=None, timezone:str='Asia/Taipei'):
    if not end: 
        end = dt.datetime.now()
    start_TimeStamp, end_TimeStamp = map(lambda x: pd.to_datetime(x).tz_localize(timezone).timestamp(), [start, end])
    url = f'https://tvapi.futures-op.com/v2/history?symbol={symbol}-1&resolution={freq}&from={start_TimeStamp:.0f}&to={end_TimeStamp:.0f}&ws=true'

    df = pd.read_json(url, convert_dates=['time']) # convert_dates is UTC
    df['datetime'] = df['time'].dt.tz_localize('UTC').dt.tz_convert(timezone) + pd.Timedelta(minutes=1)

    df_cleaned = (
        df.set_index('datetime')[['open', 'high', 'low', 'close', 'volume']]
        # .rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'TotalVolume'})
        # [['Open', 'High', 'Low', 'Close', 'TotalVolume']]
        .astype(float)
        .sort_index()
    )

    return df_cleaned


In [5]:
start = '2018-11-1'   
end = '2023-11-11'
freq = 1 #(分鐘)

In [ ]:
# NQ, ES, YM

start, end = '2023-6-1', '2023-11-11 23:59:59'   
freq = 1
symbol = 'NQ'
df = tvapi(symbol, start, end, freq)

In [154]:
Check_data(df)

unique_index? True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213634 entries, 0 to 1213633
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype                      
---  ------       --------------    -----                      
 0   DateTime     1213634 non-null  datetime64[ns, Asia/Taipei]
 1   Open         1213634 non-null  float64                    
 2   High         1213634 non-null  float64                    
 3   Low          1213634 non-null  float64                    
 4   Close        1213634 non-null  float64                    
 5   TotalVolume  1213634 non-null  float64                    
dtypes: datetime64[ns, Asia/Taipei](1), float64(5)
memory usage: 55.6 MB
None


In [155]:
df.to_pickle(f"{os.getcwd()}/../Data/NQ(1T)_2020.6.1~2023.11.11.pkl")

#### Update

In [55]:
file_name = 'NQ(1T)_2009.4.21~2023.11.11.pkl'
df_ori = pd.read_pickle(os.path.join(path_data_base, file_name))

In [56]:
df_ori.index[-1]

Timestamp('2023-11-10 16:00:00-0600', tz='US/Central')

In [47]:
timezone = 'US/Central'
# NQ, ES, YM

start, end = '2023-11-10', None
freq = 1
symbol = 'NQ'
df = tvapi(
    symbol = symbol, 
    start = start, 
    end = end, 
    freq = freq, 
    timezone = timezone,
)

In [81]:
df_update = pd.concat([df_ori, df], axis='rows').sort_index()
index_col = df_update.index.name

df_update = df_update.reset_index().drop_duplicates(
    subset=[index_col], keep='last').set_index([index_col])

In [90]:
check_data(df_update)

unique_index? True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857044 entries, 0 to 4857043
Data columns (total 6 columns):
 #   Column    Dtype                     
---  ------    -----                     
 0   datetime  datetime64[ns, US/Central]
 1   open      float64                   
 2   high      float64                   
 3   low       float64                   
 4   close     float64                   
 5   volume    float64                   
dtypes: datetime64[ns, US/Central](1), float64(5)
memory usage: 222.3 MB
None


In [93]:
end_date = df_update.index[-1].strftime('%Y.%m.%d')
file_name = f"NQ(1T)_2009.4.21~{end_date}.pkl"

df_update.to_pickle(os.path.join(path_data_base, file_name))

### TradingView

In [8]:
from tvDatafeed import TvDatafeed, Interval

username = 'a12222421@gmail.com'
password = username

tv = TvDatafeed(username, password)

In [6]:
TvDatafeed?

Init signature: TvDatafeed(username: str = None, password: str = None) -> None
Docstring:      <no docstring>
Init docstring:
Create TvDatafeed object

Args:
    username (str, optional): tradingview username. Defaults to None.
    password (str, optional): tradingview password. Defaults to None.
File:           c:\users\hottari\anaconda3\envs\env3.9\lib\site-packages\tvdatafeed\main.py
Type:           type
Subclasses:     TvDatafeedLive

In [ ]:
search = tv.search_symbol('FANG')
[ i for i in search if i['type']=='index']

In [9]:
fang = tv.get_hist(symbol='NYFANG',exchange='ICEUS',interval=Interval.in_1_hour,n_bars=1000000)
fang.index = fang.index.tz_localize('Asia/Taipei').tz_convert('US/Central')


In [ ]:
start, end = fang.index[0].date().strftime('%Y.%m.%d'), fang.index[-1].date().strftime('%Y.%m.%d')
file_name = f'fang(1h)_{start}~{end}.pkl'
fang.to_pickle(os.path.join(path_data_base, file_name))

### 期交所

In [2]:
# 買賣權分計
def 買賣權分計(Date:str)->pd.DataFrame :
    url = "https://www.taifex.com.tw/cht/3/callsAndPutsDate"
    req = requests.post(
        url,
        {
            "queryType": 1,
            "doQuery": 1,
            "queryDate": Date,
        },
    )
    df = pd.read_html(req.text)[3]
    df = df[[1, 2, 3, 14, 15]]
    df.columns = ['商品名稱', '權別', '身分別', '淨口數(權)', '淨額(權)']
    df = df.iloc[3:, :]
    # 數值化
    for col_name in df.columns[df.columns.to_list().index('身分別')+1:]:  df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
    # 數值化->to_numeric  ( errors='coerce -> 轉不過就填NaN )
    
    df['Date'] = pd.to_datetime(Date)      # df.insert(0, 'Date', Date)
    return df.set_index(['Date', '商品名稱', '權別', '身分別']).sort_index()


# 期貨未平倉
def 期貨未平倉(Date:str)->pd.DataFrame :
    url = "https://www.taifex.com.tw/cht/3/futContractsDate"
    req = requests.post(
        url,
        {
            "queryType": 1,
            "doQuery": 1,
            "queryDate": Date,
        },
    )
    df = pd.read_html(req.text)[3]
    df = df[[1, 2, 13, 14]]
    df.columns = ['商品名稱', '身分別', '淨口數(期)', '淨額(期)']
    df = df.iloc[3:, :]
    # 數值化
    for col_name in df.columns[df.columns.to_list().index('身分別')+1:]:  df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
    
    df['Date'] = pd.to_datetime(Date) 
    return df.set_index(['Date', '商品名稱', '身分別'])

In [3]:
Date_list = pd.date_range(start="2022/12/14", end="2022/12/16", freq='D')
選擇分計_new_list = []
期貨未平_new_list = []
for Date in Date_list: 
    try:
        Date = Date.strftime('%Y/%m/%d') 
        選擇分計_new_list.append(買賣權分計(Date))
        期貨未平_new_list.append(期貨未平倉(Date))
    except: continue

In [7]:
選擇分計_new = pd.concat(選擇分計_new_list, axis='rows')
期貨未平_new = pd.concat(期貨未平_new_list, axis='rows')

選擇分計 = pd.read_pickle(path+'/Data/選擇分計-2019.11.11~2022-12-14.pkl')
期貨未平 = pd.read_pickle(path+'/Data/期貨未平-2019.11.11~2022-12-14.pkl')

In [9]:
def 檢查重複index(df:pd.DataFrame):
    df = df.reset_index()
    index_col = list( df.columns[:list(df.columns).index('身分別')+1 ])  # [a:b] -> index a 到 第 b 個
    df = df.set_index( index_col )

    if not df.index.is_unique:
        return df.reset_index().drop_duplicates(subset=index_col, keep='last').set_index(index_col).sort_index()
        # drop_duplicates -> https://ithelp.ithome.com.tw/articles/10272122

    else: return df.reset_index().set_index(index_col).sort_index()

In [10]:
選擇分計_更新 = 檢查重複index(pd.concat([選擇分計, 選擇分計_new], axis='rows'))
期貨未平_更新 = 檢查重複index(pd.concat([期貨未平, 期貨未平_new], axis='rows'))

print(選擇分計_更新.reset_index().info(), 期貨未平_更新.reset_index().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22860 entries, 0 to 22859
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    22860 non-null  datetime64[ns]
 1   商品名稱    22860 non-null  object        
 2   權別      22860 non-null  object        
 3   身分別     22860 non-null  object        
 4   淨口數(權)  22860 non-null  int64         
 5   淨額(權)   22860 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42741 entries, 0 to 42740
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    42741 non-null  datetime64[ns]
 1   商品名稱    42741 non-null  object        
 2   身分別     42741 non-null  object        
 3   淨口數(期)  42741 non-null  int64         
 4   淨額(期)   42741 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory u

In [13]:
選擇分計_更新.to_pickle(path+'/Data/'+f"選擇分計-2019.11.11~{dt.date.today().strftime('%Y.%m.%d')}.pkl")
期貨未平_更新.to_pickle(path+'/Data/'+f"期貨未平-2019.11.11~{dt.date.today().strftime('%Y.%m.%d')}.pkl")

## 選股

### TEJ、GoodInfo

In [608]:
import requests
import pandas as pd
import numpy as np
import os
path = os.getcwd()
import datetime as dt

import tejapi as tej  # 888/月 才有3率
from bs4 import BeautifulSoup as BS

#### 函式區

In [2]:
# +-+-+-+-+-+-+ 代號配對 +-+-+-+-+-+-+
def get_match(need, compare):
    match = []
    delete = None
    filter = compare.copy()
    for i in need:
        try: filter.remove(delete)  # 已配對的拿掉
        except: None
        
        for j in filter:
            if i == j:
                match.append(j)
                delete = j
                break
    return match


# +-+-+-+-+-+-+ 篩選結果 +-+-+-+-+-+-+
def get_filter_code(url):
    # 解析 html
    headers = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }  # 設定 headers 的 User-Agent -> https://marketingliveincode.com/?p=4556 (偽瀏覽器?)

    req = requests.get(url, headers=headers)
    html = BS(req.content, "lxml")  # "lxml"解析比"html.parser"快 -> https://ithelp.ithome.com.tw/articles/10186119
                                    # pd.read_html(req.content)讀更久
    # 存代號
    code_list = []
    for i in html.find_all("a", class_="link_black", target="_blank"):
        try: code_list.append(int(str(i)[-8:-4]))
        except: continue
    code = code_list[int(len(code_list) / 2) :]  # 會重複抓
    return code

In [8]:
new  = list(pd.read_csv(path + "/new.csv")['代號'].astype(str))
futures = list(pd.read_csv(path + "/futures.csv")['一般'].astype(str))

In [13]:
new_OK = get_match(futures, new)
new_OK

['3035', '3037']

In [17]:
url_long = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=&FL_VAL_E3=10&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B2%B7%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=%E7%8F%BE%E9%87%91%E6%AE%96%E5%88%A9%E7%8E%87+%28%25%29&FL_VAL_S5=6&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short000 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short011 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short101 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short110 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short100 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short010 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short001 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
# "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=%E6%9C%AC%E7%9B%8A%E6%AF%94+%28PER%29&FL_VAL_S3=30&FL_VAL_E3=&FL_ITEM4=%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B3%A3%E6%97%A5%E6%95%B8%E2%80%93%E5%A4%96%E8%B3%87&FL_VAL_S4=3&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E7%94%9F%E6%8A%80%E9%86%AB%E7%99%82%E6%A5%AD&FL_RULE_CHK0=T&FL_RULE1=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E8%88%AA%E9%81%8B%E6%A5%AD&FL_RULE_CHK1=T&FL_RULE2=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%8A%80%E8%A1%8C%E6%A5%AD&FL_RULE_CHK2=T&FL_RULE3=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E4%BF%9D%E9%9A%AA%E6%A5%AD&FL_RULE_CHK3=T&FL_RULE4=%E7%94%A2%E6%A5%AD%E9%A1%9E%E5%88%A5%7C%7C%E9%87%91%E6%8E%A7%E6%A5%AD&FL_RULE_CHK4=T&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"

#### 操作區
* 分開打才不會被 GoodInfo 擋爬蟲

In [23]:
long = np.array(get_filter_code(url_long)).astype(str).tolist()          # 3升
short000 = np.array(get_filter_code(url_short000)).astype(str).tolist()  # 3降

long_111  = get_match(long, new_OK)
short_000 = get_match(short000, new_OK)

In [22]:
short011 = np.array(get_filter_code(url_short011)).astype(str).tolist()  
short101 = np.array(get_filter_code(url_short101)).astype(str).tolist()  
short110 = np.array(get_filter_code(url_short110)).astype(str).tolist()  

short_011 = get_match(short011, new_OK)
short_101 = get_match(short101, new_OK)
short_110 = get_match(short110, new_OK)

In [21]:
short100 = np.array(get_filter_code(url_short100)).astype(str).tolist() 
short010 = np.array(get_filter_code(url_short010)).astype(str).tolist()  
short001 = np.array(get_filter_code(url_short001)).astype(str).tolist() 

short_100 = get_match(short100, new_OK)
short_010 = get_match(short010, new_OK)
short_001 = get_match(short001, new_OK)

In [24]:
# 檢查有沒有漏掉
print(
    f"多:{long_111}\n"
    f"空000:{short_000}\n"
    f"空011:{short_011}\n"
    f"空101:{short_101}\n"
    f"空110:{short_110}\n"
    f"空100:{short_100}\n"
    f"空010:{short_010}\n"
    f"空001:{short_001}\n"
    )

多:['1402', '2303', '2313', '2317', '2330', '2454', '2603', '3037', '3406', '3443', '5483', '6488', '8046']
空000:['2002', '2353', '2376', '2409', '2455', '2474', '2492', '2610', '3034', '3105', '3481']
空011:['1605', '2337', '2344', '2609', '2615', '6271']
空101:['3035']
空110:[]
空100:['2327', '3008']
空010:['2408', '2498']
空001:[]



In [11]:
df_ = pd.concat([
    pd.DataFrame({'多':long_111}),
    pd.DataFrame({"空000":short_000}),
    pd.DataFrame({"空011":short_011}),
    pd.DataFrame({"空101":short_101}),
    pd.DataFrame({"空110":short_110}),
    pd.DataFrame({"空100":short_100}),
    pd.DataFrame({"空010":short_010}),
    pd.DataFrame({"空001":short_001})], axis=1)

df_.to_csv(path + f"/df_{dt.date.today()}.csv", encoding = 'utf_8_sig') # 繁中 -> utf_8_sig

### GoodInfo 篩選
* 會擋連續訪問

In [108]:
import requests
from bs4 import BeautifulSoup as BS
import os
import pandas as pd
import numpy as np

#### 函式區

In [217]:
# +-+-+-+-+-+-+ 篩選結果 +-+-+-+-+-+-+
def get_filter_code(url):
    # 解析 html
    headers = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }  # 設定 headers 的 User-Agent -> https://marketingliveincode.com/?p=4556 (偽瀏覽器?)

    req = requests.get(url, headers=headers)
    html = BS(req.content, "lxml")  # "lxml"解析比"html.parser"快 -> https://ithelp.ithome.com.tw/articles/10186119
                                    # pd.read_html(req.content)讀更久
    # 存代號
    code_list = []
    for i in html.find_all("a", class_="link_black", target="_blank"):
        try: code_list.append(int(str(i)[-8:-4]))
        except: continue
    code = code_list[int(len(code_list) / 2) :]  # 會重複抓
    return code

#### 操作區
* ndarray速度：https://blog.csdn.net/xuebinding/article/details/100194087

In [110]:
path = os.getcwd()
futures = list(pd.read_csv(path + "/futures.csv")["一般"])

In [218]:
url_long = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short000 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short011 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short101 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short110 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short100 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=0&FL_VAL_E0=&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short010 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=0&FL_VAL_E1=&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=&FL_VAL_E2=0&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
url_short001 = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E5%96%AE%E6%9C%88%E7%87%9F%E6%94%B6%E5%B9%B4%E5%A2%9E%E6%B8%9B%E7%8E%87%28%25%29&FL_VAL_S0=&FL_VAL_E0=0&FL_ITEM1=%E5%96%AE%E5%AD%A3%E2%80%93%E6%AF%9B%E5%88%A9%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S1=&FL_VAL_E1=0&FL_ITEM2=%E5%96%AE%E5%AD%A3%E2%80%93%E7%87%9F%E6%A5%AD%E5%88%A9%E7%9B%8A%E7%8E%87%E5%B9%B4%E5%A2%9E%E6%B8%9B%E6%95%B8&FL_VAL_S2=0&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E8%87%AA%E8%A8%82%E6%AC%84%E4%BD%8D_%E8%A7%80%E5%AF%9F&FL_SHEET2=&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"



long = get_filter_code(url_long)  # 3升
short000 = get_filter_code(url_short000)  # 3降
# short011 = get_filter_code(url_short011)  # -++
# short101 = get_filter_code(url_short101)  # +-+
# short110 = get_filter_code(url_short110)  # ++-
# short100 = get_filter_code(url_short100)  # +--
# short010 = get_filter_code(url_short010)  # -+-
# short001 = get_filter_code(url_short001)  # --+

match_long     = get_match(futures, long)
match_short000 = get_match(futures, short000)


In [11]:
print(f"3升:{match_long}\n3降:{match_short000}\n")

3升:[1319, 1402, 1476, 1605, 2027, 2049, 2201, 2301, 2303, 2313, 2317, 2330, 2338, 2345, 2454, 2603, 2609, 2618, 3037, 3189, 3231, 3380, 3406, 3443, 3532, 3711, 4938, 4958, 5009, 5347, 5425, 5483, 6182, 6257, 6271, 6282, 6488, 6770, 8046, 8069]
3降:[2002, 2014, 2340, 2353, 2354, 2376, 2377, 2379, 2455, 2458, 2474, 2492, 3034, 3105, 6116, 6213, 6274, 8044, 8086]



### 公開資訊觀測站
* 會擋連續訪問
* XBRL平台：https://mops.twse.com.tw/mops/web/t203sb01

In [1]:
import requests
import pandas as pd
import numpy as np
import os

In [156]:
import time
import random

path = os.getcwd()
futures = pd.read_csv(path + "/futures.csv")  # 讀繁中 -> encoding='big5', 轉字串 -> dtype=str

two_rate = []
year = 2022
season = 2

for code in list(futures["一般"]):
    print(code)
    two_rate.append(get_rate(code, year, season))
two_rate

1101
1216


KeyboardInterrupt: 

#### 2率

In [50]:
# +-+-+-+-+-+-+ 找是否在追蹤商品 +-+-+-+-+-+-+
def get_match(need, compare):
    match = []
    delete = None
    filter = compare.copy()
    for i in need:
        try:
            filter.remove(delete)  # 已配對的拿掉
        except:
            None
        for j in filter:
            if i == j:
                match.append(j)
                delete = j
                break
    return match


# +-+-+-+-+-+-+ 抓營益資料 +-+-+-+-+-+-+
def get_rate(year, season, need):
    url = "https://mops.twse.com.tw/mops/web/t163sb06"
    req = requests.post(
        url,
        {
            "year": year - 1911,
            "season": season,
            "TYPEK": "all",
            "firstin": 1,  # F12看到的  沒打會沒東西QQ
            "step": 1,  # 沒打也可以耶!
        },
    )
    df_list = pd.read_html(req.text)
    data = pd.concat(
        [i for i in df_list if i.shape[1] == 7]
    )  # 剛好「7列」才是數據表格 (shape -> (行, 列) )

    data.columns = data.iloc[0]  # 改正確 column 標題
    data = data[data["公司代號"] != "公司代號"].set_index(["公司代號"])  # 刪除標題跑進去的 row
    data.index = data.index.astype(int)

    code = get_match(need, list(data.index))
    two_rate = data.loc[code][["毛利率(%)(營業毛利)/(營業收入)", "營業利益率(%)(營業利益)/(營業收入)"]]
    two_rate.columns = [f"毛({year}Q{season})", f"利({year}Q{season})"]

    return two_rate


In [51]:
year = 2022
season = 2

path = os.getcwd()
futures = pd.read_csv(path + "/futures.csv")  # 讀繁中 -> encoding='big5', 轉字串 -> dtype=str
need = list(futures["一般"])

url = "https://mops.twse.com.tw/mops/web/t163sb06"
req = requests.post(
    url,
    {
        "year": year - 1911,
        "season": season,
        "TYPEK": "all",
        "firstin": 1,  # F12看到的  沒打會沒東西QQ
        "step": 1,  # 沒打也可以耶!
    },
)
df_list = pd.read_html(req.text)
data = pd.concat(
    [i for i in df_list if i.shape[1] == 7]
)  # 剛好「7列」才是數據表格 (shape -> (行, 列) )

data.columns = data.iloc[0]  # 改正確 column 標題
data = data[data["公司代號"] != "公司代號"].set_index(["公司代號"])  # 刪除標題跑進去的 row
data.index = data.index.astype(int)

code = get_match(need, list(data.index))
two_rate = data.loc[code][["毛利率(%)(營業毛利)/(營業收入)", "營業利益率(%)(營業利益)/(營業收入)"]]
two_rate.columns = [f"毛({year}Q{season})", f"利({year}Q{season})"]


In [85]:
path = os.getcwd()
futures = pd.read_csv(path + "/futures.csv")  # 讀繁中 -> encoding='big5', 轉字串 -> dtype=str
need = list(futures["一般"])

year = 2022
season = 2
backward = 2

rate = []
# for i in range(backward+1): rate.setdefault(f"{year-i}Q{season}", get_rate(year-i, season, need))
for i in range(backward + 1):
    rate.append(get_rate(year - i, season, need))

rate_2 = pd.concat([rate[0], rate[1], rate[2]], axis=1)
rate_2.dropna()
rate_2


,毛(2022Q2),利(2022Q2),毛(2021Q2),利(2021Q2),毛(2020Q2),利(2020Q2)
公司代號,,,,,,
1101,4.38,-3.74,28.25,22.49,31.77,26.29
1216,32.33,5.92,33.89,6.14,34.71,7.17
1319,21.55,7.10,19.64,3.81,18.86,1.51
1402,21.20,7.21,20.69,6.26,18.89,4.14
1476,27.40,20.19,28.24,20.08,26.51,15.84
...,...,...,...,...,...,...
8046,38.11,34.18,23.86,19.75,12.73,8.13
8069,47.60,25.32,45.60,15.15,44.05,6.35
8086,21.01,4.86,28.50,19.87,32.41,23.72


#### 營收
* https://lglbengo.wordpress.com/2020/03/11/%E8%87%AA%E5%AD%B8%E8%B2%A1%E5%A0%B1%E7%88%AC%E8%9F%B2%E7%AC%AC1%E7%AB%A0-%E6%8A%93%E4%B8%8A%E5%B8%82%E6%AB%83%E5%85%AC%E5%8F%B8%E9%81%8E%E5%8E%BB12%E5%80%8B%E6%9C%88%E7%87%9F%E6%94%B6/

In [ ]:
https://mops.twse.com.tw/nas/t21/sii/t21sc03_110_3_0.html

In [2]:
market = "sii"  # {'上市':'sii', '上櫃':'otc', '興櫃':'rotc', '公開發行':'pub'}
if_KY = 0  # {'是':1, '否':0}
year = 2022
month = 8
url = f"https://mops.twse.com.tw/nas/t21/{market}/t21sc03_{year-1911}_{month}_{if_KY}.html"
req = requests.get(url)
req.encoding = "big5"

dfs = pd.read_html(req.text)


In [4]:
df = pd.concat([i for i in dfs if i.shape[1] == 11])  # 剛好「11列」才是數據表格 (shape -> (行, 列) )
df.columns = df.columns.droplevel(0)  # 去掉列標題第一層 (類別)
df = df[df["公司代號"] != "合計"]  # 去掉有"合計"的整欄
df = df.set_index(["公司代號"])
df

,公司名稱,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%),備註
公司代號,,,,,,,,,,
1101,台泥,10689860,10102468,8964476,5.81,19.24,68937060,67307812,2.42,"以上營收不包含停業部門營收:111年8月 0 仟元暨110年8月595,903仟元；111年..."
1102,亞泥,7069221,7864622,7193266,-10.11,-1.72,59102719,54657145,8.13,-
1103,嘉泥,182371,178825,166022,1.98,9.84,1388559,1453918,-4.49,-
1104,環泥,605512,580420,431224,4.32,40.41,4466788,3829973,16.62,-
1108,幸福,390053,326691,249977,19.39,56.03,2599544,2635052,-1.34,去年同期本公司石材減產，致本月與去年同期相較增加50%以上。
...,...,...,...,...,...,...,...,...,...,...
9941,裕融,3201093,3093859,2732555,3.46,17.14,23639824,20970817,12.72,-
9942,茂順,352730,349111,336002,1.03,4.97,2601215,2512658,3.52,-
9944,新麗,179660,231139,178668,-22.27,0.55,1877352,1848775,1.54,-
